In [1]:
import pandas as pd
import numpy as np 
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors


import scipy.sparse
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import LabelEncoder
import joblib
from tqdm import tqdm

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [3]:
ratings = pd.read_pickle('./data/review.pickle')
ratings.point.replace({0: 10}, inplace=True)

In [5]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452459 entries, 0 to 452458
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   user_id        452459 non-null  int64 
 1   movie_id       452459 non-null  int64 
 2   movie_tile     452459 non-null  object
 3   movie_comment  452459 non-null  object
 4   point          452459 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 17.3+ MB


In [11]:
ratings.shape[0]

452459

In [10]:
k = 113115
k * 4

452460

In [12]:
df = ratings.copy()

In [13]:
k = 113115  # 1DataFrameあたりの行数
dfs = [df.loc[i:i+k-1, :] for i in range(0, len(df), k)]

In [14]:
dfs

[        user_id  movie_id                          movie_tile  \
 0         20001     25942                   シン・エヴァンゲリオン劇場版：||   
 1         20001     26788  僕のヒーローアカデミアTHEMOVIE　ワールドヒーローズミッション   
 2         20001     25605                             メランコリック   
 3         20001     24933                       ドラゴンボール超／ブロリー   
 4         20001     23973                          ReLIFEリライフ   
 ...         ...       ...                                 ...   
 113110    20981     16194                               おくりびと   
 113111    20981     15548                             崖の上のポニョ   
 113112    20981     16353                              1408号室   
 113113    20981     16744                              ウォッチメン   
 113114    20981     15749                        ダークナイト(2008)   
 
                                             movie_comment  point  
 0       劇場で見てさらにAmazonPrimeで再確認しました。点数は劇場版で見た際のメモの点数のま...      8  
 1       空いてそうな時間帯を狙っていきました。時系列的には連続アニメの最新話のあとぐらいの話みたいで...      8  
 2

In [18]:
fname

'./data_reviews/review3.pickle'

In [17]:
for i, df_i in enumerate(dfs):
    fname = './data_reviews/review' + str(i) + ".pickle"
    df_i.to_pickle(fname)

In [20]:
review0_df = pd.read_pickle('./data_reviews/review0.pickle')
review1_df = pd.read_pickle('./data_reviews/review1.pickle')
review2_df = pd.read_pickle('./data_reviews/review2.pickle')
review3_df = pd.read_pickle('./data_reviews/review3.pickle')

In [21]:
review0_df.head()

,user_id,movie_id,movie_tile,movie_comment,point
0,20001,25942,シン・エヴァンゲリオン劇場版：||,劇場で見てさらにAmazonPrimeで再確認しました。点数は劇場版で見た際のメモの点数のま...,8
1,20001,26788,僕のヒーローアカデミアTHEMOVIE ワールドヒーローズミッション,空いてそうな時間帯を狙っていきました。時系列的には連続アニメの最新話のあとぐらいの話みたいで...,8
2,20001,25605,メランコリック,AmazonPrimeで見ました。調べると製作費３００万だそうで。それもあって初めてみる俳優...,8
3,20001,24933,ドラゴンボール超／ブロリー,なお、ドラゴンボールの映画は「神と神」と「復活のF」ぐらいしか見たことがないのでオリジナルの...,6
4,20001,23973,ReLIFEリライフ,漫画とアニメ（＋OVA）は全部見てます。で、漫画原作は222話で10冊以上あるのでやもえない...,6


In [22]:
review1_df.head()

,user_id,movie_id,movie_tile,movie_comment,point
113115,20981,11244,バットマンビギンズ,ダークナイトの序章として観るべき。ヒーローって大変なんだね、と思える一本。,7
113116,20981,16282,Ｐ.Ｓ.アイラヴユー,起◎承◎転？結×。男としては感情移入できまへん。,3
113117,20981,14637,３００＜スリーハンドレッド＞,実写版「男塾」。,6
113118,20981,16782,ヱヴァンゲリヲン新劇場版：破,テレビ版を観た・観ていない次第で点が割れそう。戦闘シーンの完成度高し。,8
113119,20981,14682,ヱヴァンゲリヲン新劇場版：序,単なる焼き直しかと思いきやさにあらず。ヤシマ作戦のとこだけでも観る価値あり。,7


In [23]:
review2_df.head()

,user_id,movie_id,movie_tile,movie_comment,point
226230,23685,13153,機動戦士ZガンダムII恋人たち,ストーリーの端折りは相変わらずのためＴＶ版未見の方は辛いかも。視聴者層を考えれば当然ですが・...,6
226231,23685,1286,シャフト,サミュエルは相手役・脇役が光ると思ってるので・・ちょっとなぁ的な部分はありました。他の方も書...,5
226232,23685,1393,コヨーテ・アグリー,劇場で未見でＤＶＤ買ったものの１年以上放置してました(笑）少し人物の掘り下げが少ない部分もあ...,7
226233,23685,10398,ボーン・スプレマシー,年末安かったので購入し自宅シアターにて鑑賞。マット・デイモンも老けたなぁ・・カーチェイスはち...,6
226234,23685,4898,デアデビル,コリン・ファレルはおもしろかった。。でもそれだけしか残らなかった(笑),5


In [24]:
review3_df.head()

,user_id,movie_id,movie_tile,movie_comment,point
339345,25489,14740,次郎長三国志 第九部 荒神山,前作からのつながりで、山に籠もってる、いう設定。土屋嘉男の農民が恨みから変わっていく展開がポ...,6
339346,25489,13660,次郎長三国志 第八部 海道一の暴れん坊,豚松の母親の嘆きなど、「カタギ」と「馬鹿」が対比される。やくざというのは、つまり「馬鹿」の開...,7
339347,25489,13704,次郎長三国志 第七部 初祝い清水港,正月映画らしい道具立てで、忠臣蔵の七段目をベースにしたような一編。もっとも大石はリコウがバカ...,7
339348,25489,13692,次郎長三国志 第六部 旅がらす次郎長一家,御詠歌を流す巡礼で始まり、一転渋い一編となる。逃げ回り野に伏す日々。石松が思わず吃らずに嘆い...,7
339349,25489,13688,次郎長三国志 第五部 殴込み甲州路,まず祭り。陽気な爆発。路地の中にゆっくりと踊りながら入り込んでくるおせんちゃんのカット。お囃...,7


In [26]:
review_df_merge = review0_df.append(review1_df).append(review2_df).append(review3_df)
review_df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452459 entries, 0 to 452458
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   user_id        452459 non-null  int64 
 1   movie_id       452459 non-null  int64 
 2   movie_tile     452459 non-null  object
 3   movie_comment  452459 non-null  object
 4   point          452459 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 17.3+ MB


In [28]:
review_df_merge.tail()

,user_id,movie_id,movie_tile,movie_comment,point
452454,27958,8188,ロード・オブ・ザ・リング／王の帰還,原作は本当に大好きな作品。その世界をここまで再現してくれた制作陣には感謝しかありません。満点...,9
452455,27960,26636,JUNKHEAD,ストップモーションについてそんなに詳しいわけではないが、その熱意に圧倒された。女性描写に鋭い...,9
452456,27960,26543,花束みたいな恋をした,瀧波ユカリさんがnoteで大絶賛していたので気になり会社を出た後映画館に滑りこんだ。端的に言...,10
452457,27961,7549,スカイ・ライダーズ,中一の時、和歌山田辺のトキワ座にて「オーメン」の併映として鑑賞。メインを凌駕するほどの面白さ...,7
452458,27973,26407,キネマの神様,木場の109シネマズで鑑賞。笑いあり、涙ありの良作。個人的には傑作といっても過言ではないと思...,9


In [29]:
ratings.tail()

,user_id,movie_id,movie_tile,movie_comment,point
452454,27958,8188,ロード・オブ・ザ・リング／王の帰還,原作は本当に大好きな作品。その世界をここまで再現してくれた制作陣には感謝しかありません。満点...,9
452455,27960,26636,JUNKHEAD,ストップモーションについてそんなに詳しいわけではないが、その熱意に圧倒された。女性描写に鋭い...,9
452456,27960,26543,花束みたいな恋をした,瀧波ユカリさんがnoteで大絶賛していたので気になり会社を出た後映画館に滑りこんだ。端的に言...,10
452457,27961,7549,スカイ・ライダーズ,中一の時、和歌山田辺のトキワ座にて「オーメン」の併映として鑑賞。メインを凌駕するほどの面白さ...,7
452458,27973,26407,キネマの神様,木場の109シネマズで鑑賞。笑いあり、涙ありの良作。個人的には傑作といっても過言ではないと思...,9


In [31]:
review_df_merge.equals(ratings)

True